# Recomendation System using Boltzmann Machine and Pytorch

## Importing Library 

In [1]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing Dataset

### 1. Movie Data

In [2]:

movies = pd.read_csv('ml-1m/movies.dat',
                     sep= '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1'
                     )
print(movies.head(10))
print("\nShape of Movies set ", movies.shape)

    0                                   1                             2
0   1                    Toy Story (1995)   Animation|Children's|Comedy
1   2                      Jumanji (1995)  Adventure|Children's|Fantasy
2   3             Grumpier Old Men (1995)                Comedy|Romance
3   4            Waiting to Exhale (1995)                  Comedy|Drama
4   5  Father of the Bride Part II (1995)                        Comedy
5   6                         Heat (1995)         Action|Crime|Thriller
6   7                      Sabrina (1995)                Comedy|Romance
7   8                 Tom and Huck (1995)          Adventure|Children's
8   9                 Sudden Death (1995)                        Action
9  10                    GoldenEye (1995)     Action|Adventure|Thriller

Shape of Movies set  (3883, 3)


### 2. User Dataset

In [3]:

user = pd.read_csv('ml-1m/users.dat',
                     sep= '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1'
                     )

print(user.head(10))
print("\nShape of user set ", user.shape)

    0  1   2   3      4
0   1  F   1  10  48067
1   2  M  56  16  70072
2   3  M  25  15  55117
3   4  M  45   7  02460
4   5  M  25  20  55455
5   6  F  50   9  55117
6   7  M  35   1  06810
7   8  M  25  12  11413
8   9  M  25  17  61614
9  10  F  35   1  95370

Shape of user set  (6040, 5)


### 3. Rating Dataset

In [4]:

rating = pd.read_csv('ml-1m/ratings.dat',
                     sep= '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1'
                     )

print(rating.head(10))
print("\nShape of rating set ", rating.shape)

   0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968
3  1  3408  4  978300275
4  1  2355  5  978824291
5  1  1197  3  978302268
6  1  1287  5  978302039
7  1  2804  5  978300719
8  1   594  4  978302268
9  1   919  4  978301368

Shape of rating set  (1000209, 4)


## Preparing Training and Test set

In [5]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')

# converting to array
training_set = np.array(training_set, dtype = int)
print("Shape of Training Set :", training_set.shape)

Shape of Training Set : (79999, 4)


In [6]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')

# converting to array
test_set = np.array(test_set, dtype = int)
print("Shape of Test Set :", test_set.shape)

Shape of Test Set : (19999, 4)


## Creating the number of user and movies

In [7]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
print(nb_users)

nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(nb_movies)

943
1682


## Converting the data into an array with users in line and movies in columns

In [8]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0]==id_users]
        id_rating = data[:, 2][data[:, 0] == id_users]
        rating = np.zeros(nb_movies)
        rating[id_movies-1] = id_rating
        new_data.append(list(rating))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)
        

## Converting the data into Torch tensor

In [9]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the rating into binary rating 1 (liked) and 0 (not liked)

In [10]:
training_set[training_set == 0 ] = -1 # Not rated by user
training_set[training_set == 1 ] = 0
training_set[training_set == 2 ] = 0
training_set[training_set >= 3 ] = 1

test_set[test_set == 0 ] = -1 # Not rated by user
test_set[test_set == 1 ] = 0
test_set[test_set == 2 ] = 0
test_set[test_set >= 3 ] = 1

## Creating architecture of the NN

In [14]:
class RBM():
    #nv : Number of movies
    #nh : Number of features (user defined)
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv) # Weights matrics for Hidden and Visible Node
        self.a = torch.randn(1, nh)  # Bias for hidden node
        self.b = torch.randn(1, nv)  # Bias for visible node
    
    def sample_h(self, x):   #x is visible node
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y): #y is hidden node
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk),0)
        self.a += torch.sum((ph0 - phk),0)
        
        

In [22]:
nv = len(training_set[0])
nh = 100
batch_size = 100

rbm = RBM(nv, nh)
print(rbm.W.shape, rbm.b.shape, rbm.a.shape)
rbm.a

torch.Size([100, 1682]) torch.Size([1, 1682]) torch.Size([1, 100])


tensor([[-1.5462,  0.7726, -0.0237,  0.2317, -2.0629, -0.6503, -1.1439,  1.3201,
         -0.7859, -0.0381, -0.0084,  0.7152, -1.6527,  0.6482, -0.2852, -1.7392,
          1.3449, -0.5556,  0.4475,  1.5154,  2.1418,  1.3434,  0.2634, -0.7262,
          0.6484, -2.0660,  1.9452, -1.0321, -0.1082, -1.2566, -0.9497,  0.5279,
         -0.8019,  0.1546, -0.2203,  0.0570, -0.6431,  0.1618, -1.4387,  0.2779,
          0.1663, -0.1109, -0.5057,  1.5321, -0.6253,  0.7436, -0.7225,  1.8034,
          0.4667, -0.1762, -1.0167, -0.8764,  1.1631,  0.4188, -0.3199,  0.1428,
          0.1303, -1.3454,  0.2416, -0.9323,  0.6393, -1.1094, -1.7462, -0.1782,
         -0.5228,  0.1020, -0.6122, -1.2958,  1.1010,  0.6858,  0.1612,  2.1209,
         -0.5224,  1.0597,  0.7066,  0.0809,  0.1546, -0.8807, -1.4025,  1.2550,
         -1.1381, -0.1660,  1.4488,  1.6292, -0.3027, -0.9113,  0.4493, -0.0310,
          0.6262, -0.6491,  0.0594,  2.9500, -0.4485,  1.2207,  1.6224,  0.4517,
         -0.0432, -0.1452, -

## Training the RBM

In [25]:
nb_epoch = 10
for epoch in range(1, nb_epoch+1):
    train_loss = 0 
    s = 0. # counter
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(15):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print(f'epoch: {epoch} loss: {train_loss/s}')
            
    

epoch: 1 loss: 0.24706202745437622
epoch: 2 loss: 0.24485741555690765
epoch: 3 loss: 0.2502173185348511
epoch: 4 loss: 0.24426428973674774
epoch: 5 loss: 0.249492347240448
epoch: 6 loss: 0.24944064021110535
epoch: 7 loss: 0.24622675776481628
epoch: 8 loss: 0.2487250119447708
epoch: 9 loss: 0.24725304543972015
epoch: 10 loss: 0.24780556559562683


# Predicting for the test set

In [26]:
test_loss = 0 
s = 0. # counter
for id_user in range(0, nb_users):
    v = training_set[id_user : id_user + 1]
    vt = test_set[id_user : id_user + 1]
    if len(vt[vt>=0]) > 0  :
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print(f'loss: {test_loss/s}')

loss: 0.2414473593235016
